In [36]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
file = pd.read_csv("housing.csv")

In [37]:
file.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   longitude           20640 non-null  float64
 1   latitude            20640 non-null  float64
 2   housing_median_age  20640 non-null  float64
 3   total_rooms         20640 non-null  float64
 4   total_bedrooms      20433 non-null  float64
 5   population          20640 non-null  float64
 6   households          20640 non-null  float64
 7   median_income       20640 non-null  float64
 8   median_house_value  20640 non-null  float64
 9   ocean_proximity     20640 non-null  object 
dtypes: float64(9), object(1)
memory usage: 1.6+ MB


In [38]:
file.ocean_proximity.unique()

array(['NEAR BAY', '<1H OCEAN', 'INLAND', 'NEAR OCEAN', 'ISLAND'],
      dtype=object)

In [39]:
#Q1
file.isnull().sum()

longitude               0
latitude                0
housing_median_age      0
total_rooms             0
total_bedrooms        207
population              0
households              0
median_income           0
median_house_value      0
ocean_proximity         0
dtype: int64

In [40]:
#Q2
file.population.median()

1166.0

In [41]:
#Q3
y = file['median_house_value']
x = file.drop(["median_house_value"], axis = 1)

In [42]:

x = pd.get_dummies(x, columns = ['ocean_proximity'])
y = np.log1p(y)

In [43]:
from sklearn.model_selection import train_test_split

In [44]:
x_train, x_t, y_train,y_t = train_test_split(x,y, train_size = 0.6, random_state = 42)
x_val, x_test, y_val, y_test = train_test_split(x_t, y_t, test_size = 0.5, random_state = 42)

In [45]:
#using mean
mean = x_train.mean()
x_train = x_train.fillna(mean)
#using zero
x_tt = x_train.fillna(0)

In [46]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(x_train, y_train)
r = LinearRegression()
r.fit(x_tt, y_train)

LinearRegression()

In [47]:
from sklearn.metrics import mean_squared_error

In [48]:
#with mean
x_test_m = x_val.fillna(mean)
y_pred1 = lr.predict(x_test_m)

#with zero
x_te = x_val.fillna(0)
y_pred2 = r.predict(x_te)

In [49]:
import math
error_mean = math.sqrt(mean_squared_error(y_pred1, y_val))
error_zero = math.sqrt(mean_squared_error(y_pred2, y_val))

In [50]:
error_mean

0.3367140807271667

In [51]:
error_zero

0.3387134808494152

In [52]:
#Q4
from sklearn.linear_model import Ridge, Lasso
alphas = [0, 0.000001, 0.0001, 0.001, 0.01, 0.1, 1, 5, 10]
for a in alphas:
    rd = Ridge(alpha = a, fit_intercept = True)
    model = rd.fit(x_tt, y_train)
    y_new = model.predict(x_te)
    error = round(math.sqrt(mean_squared_error(y_new, y_val)),4)
    print("alpha {} = RMSE{}".format(a,error))

alpha 0 = RMSE0.3387
alpha 1e-06 = RMSE0.3387
alpha 0.0001 = RMSE0.3387
alpha 0.001 = RMSE0.3387
alpha 0.01 = RMSE0.3387
alpha 0.1 = RMSE0.3387
alpha 1 = RMSE0.3388
alpha 5 = RMSE0.3389
alpha 10 = RMSE0.339


C:\Users\user\anaconda3\anaconda3\lib\site-packages\sklearn\linear_model\_ridge.py:147: LinAlgWarning: Ill-conditioned matrix (rcond=9.58843e-24): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True,
C:\Users\user\anaconda3\anaconda3\lib\site-packages\sklearn\linear_model\_ridge.py:147: LinAlgWarning: Ill-conditioned matrix (rcond=9.80708e-18): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True,


In [53]:
seed_errors = []
for i in range(10):
    x_train, x_t, y_train,y_t = train_test_split(x,y, train_size = 0.6, random_state = i)
    x_val, x_test, y_val, y_test = train_test_split(x_t, y_t, test_size = 0.5, random_state = i)
    x_tee = x_val.fillna(0)
    y_predd = r.predict(x_tee)
    error_z = math.sqrt(mean_squared_error(y_predd, y_val))
    seed_errors.append(error_z)
seed_errors

[0.3270416806809724,
 0.33418959662124914,
 0.33337656564093643,
 0.3231000368075514,
 0.3257060580063175,
 0.32866241707691496,
 0.3268410937211536,
 0.33466032186306105,
 0.3314297894126479,
 0.323372750894921]

In [54]:
STD = np.std(seed_errors)

In [56]:
round(STD,3)

0.004

In [58]:
#Q6
x2 = x.fillna(0)
x_train, x_test, y_train,y_test = train_test_split(x2,y, train_size = 0.6, random_state = 9)
rd = Ridge(alpha = 0.001, fit_intercept = True)
rd.fit(x_train, y_train)
y_p = rd.predict(x_test)
math.sqrt(mean_squared_error(y_p, y_test))

0.32903858858955526